In [1]:
#!pip install transformers
#!pip install keybert
#pip install auto_gptq==0.2.0
import torch
import pandas as pd
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor
import time
import ollama
import fitz  # PyMuPDF
import os

print(torch.cuda.is_available())

d:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


True


In [5]:
# zero-shot learning pipeline, summarization models available on huggingface - https://huggingface.co/models?pipeline_tag=summarization&sort=downloads

from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1

summarizer_facebook_bart = pipeline("summarization", model="facebook/bart-large-cnn",device=device) # https://huggingface.co/facebook/bart-large-cnn
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",device=device))
summarizer_philschmid_bart = pipeline("summarization", model="philschmid/bart-large-cnn-samsum",device=device) # https://huggingface.co/philschmid/bart-large-cnn-samsum
summarizer_google_pegasus = pipeline("summarization", model="google/pegasus-xsum",device=device) # https://huggingface.co/google/pegasus-xsum
summarizer_falconsai = pipeline("summarization", model="Falconsai/text_summarization",device=device) # https://huggingface.co/Falconsai/text_summarization
summarizer_distilbart = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6",device=device)
# summarizer_google_pegasusL = pipeline("summarization", model="google/pegasus-large",device=device)) # https://huggingface.co/google/pegasus-large
# summarizer_sshleifer_distilbart_6_6 = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6",device=device)) # https://huggingface.co/sshleifer/distilbart-cnn-6-6
summarizer_azma_bart = pipeline("summarization", model="Azma-AI/bart-large-text-summarizer",device=device) # https://huggingface.co/Azma-AI/bart-large-text-summarizer

list_of_summarizer = [summarizer_facebook_bart,summarizer_falconsai,summarizer_google_pegasus,summarizer_azma_bart,summarizer_distilbart,summarizer_philschmid_bart]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
if torch.cuda.is_available(): device = 0  
else: device = -1  
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [ ]:
# #testing with llama13B
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers import GPTQConfig

device = 0 if torch.cuda.is_available() else -1
quantization_config = GPTQConfig(bits=4, disable_exllama=True)

model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/WizardLM-1.0-Uncensored-Llama2-13B-GPTQ",
    quantization_config=quantization_config
).to('cuda')

tokenizer = AutoTokenizer.from_pretrained("TheBloke/WizardLM-1.0-Uncensored-Llama2-13B-GPTQ")

# Create the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)


LLM summarization model (pretrained)

In [9]:
description = """principal activity is that of a the following items are recognised in OCI: property developer and the place of business is in Singapore. The Group is also a 20% (2021: 20%) partner in Park Court Aoyama The Tower,
a joint arrangement formed with a third party, whose principal activity is that of a   differences arising on the translation of monetary items that in substance form part of the Group s net property developer and the place of business is in Japan.
The Group has classified both joint arrangements as joint investment in a foreign operation; operations as the joint venture partners control the joint arrangements collectively and the joint arrangements are not structured through separate legal vehicles.
   an investment in equity securities designated at fair value through other comprehensive income (FVOCI); (vi) Transactions eliminated on consolidation
   a financial liability designated as a hedge of the net investment in a foreign operation to the extent that the hedge is effective; or Intra-group balances and transactions, and any unrealised income or expenses arising from intra-group transactions,
   are eliminated in preparing the consolidated financial statements. Unrealised gains arising from   qualifying cash flow hedges to the extent that the hedges are effective. transactions with equity-accounted investee are eliminated against the investment
    to the extent of the Group s interest in the investee. Unrealised losses are eliminated in the same way as unrealised gains, but only to the (ii) Foreign operations extent that there is no evidence of impairment. The assets and liabilities of foreign
    operations, including goodwill and fair value adjustments arising on acquisition, (vii) Subsidiaries, associates and joint ventures in the separate financial statements are translated to Singapore dollars at exchange rates prevailing at the reporting
    date. The income and expenses of foreign operations are translated to Singapore dollars at exchange rates at the dates of the transactions. Investments in subsidiaries, associates and joint ventures are stated in the Company s statement of financial
    position at cost less accumulated impairment losses.
Foreign currency differences are recognised in OCI. However, if the foreign operation is a non wholly-owned subsidiary, then the relevant proportionate share of the translation difference is allocated to the non-controlling 3."""
              
#Summarize the description
summary = summarizer_facebook_bart(description, max_length=150, min_length=30, do_sample=False)
print(summary[0]['summary_text'])

from keybert import KeyBERT
kw_model = KeyBERT()

# Extract keywords from the summarized description
keywords = kw_model.extract_keywords(summary[0]['summary_text'])
print(keywords)

d:\Anaconda\lib\site-packages\transformers\models\bart\modeling_bart.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


The Group has classified both joint arrangements as joint investment in a foreign operation. The Group is also a 20% (2021: 20%) partner in Park Court Aoyama The Tower, a joint arrangement formed with a third party.
[('joint', 0.4634), ('aoyama', 0.3924), ('arrangement', 0.357), ('partner', 0.3238), ('arrangements', 0.3205)]


In [24]:
#BRC ASIA LIMITED 2022
#Principal Activity SSIC Code 24109
#MANUFACTURE OF BASIC IRON AND STEEL N.E.C. (INCLUDING SMELTING)

# description = """principal activities of the Company are the prefabrication of steel reinforcement for use in concrete, 
# trading of steel reinforcing bars, and manufacturing and sale of wire mesh fences. 
# The principal activities of the subsidiaries are disclosed in Note 13 to the fi nancial statements. 
# """

description = """principal activities are in the retail and distribution of luxury timepieces and luxury accessories. The group holds inventories of $,, (: $,,) as at the end of the reporting year. The cost of inventories may not be recoverable in full if any inventories are damaged, or if they become obsolete, or if their selling prices have declined. The estimate of allowance for inventories is based on the age of these inventories, prevailing market conditions of the luxury timepieces and related products in the retail industry and historical allowance experience which require management s judgement. Management applies judgement in determining the appropriate allowance for inventories based upon a detailed technical assessment of inventories concerned including considering the future demand and future selling prices for the products and ageing analysis of inventories. This methodology relies upon assumptions made in determining the appropriate allowance percentages for each category of inventories.
"""

#CITY DEVELOPMENTS LIMITED 
#Principal Activity SSIC Code 68101 REAL ESTATE DEVELOPERS
# description = """principal activity is that of a the following items are recognised in OCI: property developer and the place of business is in Singapore. The Group is also a 20% (2021: 20%) partner in Park Court Aoyama The Tower,
# a joint arrangement formed with a third party, whose principal activity is that of a   differences arising on the translation of monetary items that in substance form part of the Group s net property developer and the place of business is in Japan.
# The Group has classified both joint arrangements as joint investment in a foreign operation; operations as the joint venture partners control the joint arrangements collectively and the joint arrangements are not structured through separate legal vehicles.
#    an investment in equity securities designated at fair value through other comprehensive income (FVOCI); (vi) Transactions eliminated on consolidation
#    a financial liability designated as a hedge of the net investment in a foreign operation to the extent that the hedge is effective; or Intra-group balances and transactions, and any unrealised income or expenses arising from intra-group transactions,
#    are eliminated in preparing the consolidated financial statements. Unrealised gains arising from   qualifying cash flow hedges to the extent that the hedges are effective. transactions with equity-accounted investee are eliminated against the investment
#     to the extent of the Group s interest in the investee. Unrealised losses are eliminated in the same way as unrealised gains, but only to the (ii) Foreign operations extent that there is no evidence of impairment. The assets and liabilities of foreign
#     operations, including goodwill and fair value adjustments arising on acquisition, (vii) Subsidiaries, associates and joint ventures in the separate financial statements are translated to Singapore dollars at exchange rates prevailing at the reporting
#     date. The income and expenses of foreign operations are translated to Singapore dollars at exchange rates at the dates of the transactions. Investments in subsidiaries, associates and joint ventures are stated in the Company s statement of financial
#     position at cost less accumulated impairment losses.
# Foreign currency differences are recognised in OCI. However, if the foreign operation is a non wholly-owned subsidiary, then the relevant proportionate share of the translation difference is allocated to the non-controlling 3."""
       
    
for summarizer in list_of_summarizer:
        model_name = summarizer.model.name_or_path if hasattr(summarizer, 'model') else summarizer.__name__
        summary = summarizer(description, max_length=max(30, int(len(description)/6)), min_length=30, do_sample=False)
        print(f"{model_name}: {summary[0]['summary_text']}")


#philschmid/bart-large-cnn-samsum - perform the best

facebook/bart-large-cnn: The cost of inventories may not be recoverable in full if any inventories are damaged, or if they become obsolete. The estimate of allowance for inventories is based on the age of these inventories.


Your max_length is set to 153, but your input_length is only 142. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=71)


Falconsai/text_summarization: the cost of inventories may not be recoverable in full if any inventories are damaged or if their selling prices have declined . The estimate of allowance for inventories is based on the age of these inventories, prevailing market conditions of the luxury timepieces and related products in the retail industry and historical allowance experience . Management applies judgement in determining the appropriate allowance based upon a detailed technical assessment .
google/pegasus-xsum: The following table sets out the allowance for inventories set out in the Management s Discussion and Analysis of Financial Condition and Results of Operations for the six months to 30 June 2014.
Azma-AI/bart-large-text-summarizer: Luxury timepieces and luxury accessories are held in inventories. Management estimates the estimated cost of the inventories based on technical assessment and market conditions.
sshleifer/distilbart-cnn-12-6:  The cost of inventories may not be recovera

In [2]:
#Q&A model
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

pipe = pipeline("question-answering", model="deepset/roberta-base-squad2")

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

context = """principal activities are in the retail and distribution of luxury timepieces and luxury accessories. 
The group holds inventories of $231,624,000 (2022: $211,199,000) as at the end of the reporting year. 
The cost of inventories may not be recoverable in full if any inventories are damaged, or if they become obsolete,
 or if their selling prices have declined. The estimate of allowance for inventories is based on the age of these inventories,
   prevailing market conditions of the luxury timepieces and related products in the retail industry and historical allowance experience which require management s judgement. 
   Management applies judgement in determining the appropriate allowance for inventories based upon a detailed technical assessment of inventories concerned including considering the future demand and future selling prices for the products and ageing analysis of inventories. This methodology relies upon assumptions made in determining the appropriate allowance percentages for each category of inventories.."""

question = "What's the principal activity of the company?"
answer = pipe(question=question, context=context)

print("Answer:", answer['answer'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Answer: retail and distribution of luxury timepieces and luxury accessories


In [15]:
#Files required for summarization
df_for_Summary=pd.read_excel("../dataSources/ScrapedOutputFiles/10) extracted_notes_pages_with_uen.xlsx")

#SSIC Ref detailes
SSIC_ref_df = pd.read_csv('SSIC Fact Ref Table.csv')
SSIC_ref_df_test=SSIC_ref_df.head(10)
#SSIC_ref_df_test
df_for_Summary

,PDF Name,Page Number,UEN Number,Notes Page Content
0,ABR HOLDINGS LIMITED 2022 (197803023H).pdf,74,197803023H,principal activities of the Company are the ma...
1,ABUNDANCE INTERNATIONAL LIMITED 2022 (19750157...,56,197501572K,principal activities of the Company are those ...
2,ABUNDANTE LIMITED 2023 (197902587H).pdf,48,197902587H,principal activities of the Company are those ...
3,ACCRELIST LTD. 2023 (201413466E).pdf,55,201413466E,principal activities of the Company is investm...
4,ACESIAN PARTNERS LIMITED 2022 (199505699D).pdf,54,199505699D,principal activities of the Group consist of d...
...,...,...,...,...
82,TELECHOICE INTERNATIONAL LIMITED 2022 (1998020...,129,199802072R,principal activities of the Company during the...
83,TT INTERNATIONAL LIMITED 2023 (198403771D).pdf,56,198403771D,principal activities of the Company are those ...
84,UNITED OVERSEAS INSURANCE LIMITED 2022 (197100...,4,197100152R,principal activities are the underwriting 1
85,VENTURE CORPORATION LIMITED 2022 (198402886H).pdf,107,198402886H,principal activities of the Company are to pro...


In [7]:
import pandas as pd
from transformers import pipeline
pipe = pipeline("question-answering", model="deepset/roberta-base-squad2",device=0)

def get_answer(row):
    context = row['Notes Page Content']
    question = "What are all the principal activities of the company? List down all the activities."  
    result = pipe(question=question, context=context)
    return result['answer']

df_for_Summary['Q&A model Output'] = df_for_Summary.apply(get_answer, axis=1)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [17]:
def dynamic_summarizer(summarizer, text, min_length=30, length_fraction=0.9, too_short_threshold=30):
    input_length = len(text.split())
    # If the text is shorter than the threshold, return original text
    if input_length < too_short_threshold:
        return text
    max_length = int(input_length * length_fraction)
    max_length = max(min_length, max_length)
    summary=summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summary

df_for_Summary['Summarized_Description_azma_bart'] = df_for_Summary['Notes Page Content'].apply(
    lambda x: dynamic_summarizer(summarizer_azma_bart, x)
)
df_for_Summary['Summarized_Description_facebook_bart'] = df_for_Summary['Notes Page Content'].apply(
    lambda x: dynamic_summarizer(summarizer_facebook_bart, x)
)
df_for_Summary['Summarized_Description_philschmid_bart'] = df_for_Summary['Notes Page Content'].apply(
    lambda x: dynamic_summarizer(summarizer_philschmid_bart, x)
)

#df_for_Summary['Length of original text'] =  df_for_Summary['Notes Page Content'].str.len()
df_for_Summary['Input_length'] = df_for_Summary['Notes Page Content'].apply(lambda x: len(x.split()))

df_for_Summary['Summarised?'] = df_for_Summary.apply(
    lambda row: 'No' if row['Notes Page Content'] == row['Summarized_Description_azma_bart'] else 'Yes',axis=1
)
# df_for_Summary['Summarized_Description_azma_bart'] = df_for_Summary['Notes Page Content'].apply(
#     lambda x: summarizer_azma_bart(x, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
# )
# df_for_Summary['Summarized_Description_facebook_bart'] = df_for_Summary['Notes Page Content'].apply(
#     lambda x: summarizer_facebook_bart(x, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
# )
# df_for_Summary['Summarized_Description_philschmid_bart'] = df_for_Summary['Notes Page Content'].apply(
#     lambda x: summarizer_philschmid_bart(x, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
# )

# 85 rows - 5m 41s

In [4]:
df_for_Summary.to_excel("D:\ssicsync\LLM_Test\Summarised_output_for_model v2.xlsx", index=False)

In [ ]:
###############################################################################################################################################
# TDIDF

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
nltk.download('stopwords')
import re
# nltk.download()
from nltk.corpus import stopwords

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    words_to_remove = ['principal','activity' ,'activities', 'investment', 'holding', 'holdings', 'singapore', 'exchange', 'securities', 'trading', 'limited', 'company']  # List of additional words to remove############################################################
    # Remove additional words
    if words_to_remove:
        text = ' '.join(word for word in text.split() if word not in words_to_remove)
    return text

df_for_Summary['Azma_bart_tfidf'] = df_for_Summary['Summarized_Description_azma_bart'].apply(preprocess_text)
df_for_Summary['FB_bart_tfidf'] = df_for_Summary['Summarized_Description_facebook_bart'].apply(preprocess_text)
df_for_Summary['Philschmid_bart_tfidf'] = df_for_Summary['Summarized_Description_philschmid_bart'].apply(preprocess_text)

# Fill NaN values in 'Azma_bart_tfidf' with 'Notes Page Content'
df_for_Summary['Azma_bart_tfidf'] = df_for_Summary.apply(
    lambda row: row['Notes Page Content'] if pd.isna(row['Azma_bart_tfidf']) else row['Azma_bart_tfidf'],
    axis=1
)

df_for_Summary['FB_bart_tfidf'] = df_for_Summary.apply(
    lambda row: row['Notes Page Content'] if pd.isna(row['FB_bart_tfidf']) else row['FB_bart_tfidf'],
    axis=1
)
df_for_Summary['Philschmid_bart_tfidf'] = df_for_Summary.apply(
    lambda row: row['Notes Page Content'] if pd.isna(row['Philschmid_bart_tfidf']) else row['Philschmid_bart_tfidf'],
    axis=1
)

tfidf_vectorizer = TfidfVectorizer(
    # # max_df=0.8,         # Ignore terms that appear in more than 80% of documents
    # # min_df=2,           # Ignore terms that appear in fewer than 2 documents
    # max_features=1000,  # Consider only the top 1000 terms by TF-IDF score
    # ngram_range=(1, 2), # Include both unigrams and bigrams
    # stop_words='english' # Remove standard English stop words
)

# Fit and transform the preprocessed text ('Notes Page Content2')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_for_Summary['Azma_bart_tfidf'])
tfidf_matrix = tfidf_vectorizer.fit_transform(df_for_Summary['FB_bart_tfidf'])  
tfidf_matrix = tfidf_vectorizer.fit_transform(df_for_Summary['Philschmid_bart_tfidf'])
tfidf_matrix = tfidf_vectorizer.fit_transform(df_for_Summary['Q&A model Output_tfIDF'])

# Get feature names (terms)
terms = tfidf_vectorizer.get_feature_names_out()

# Set a TF-IDF threshold
tfidf_threshold = 0.1

# Function to get important terms from a document
def get_important_terms(doc_index, tfidf_matrix, terms, threshold, top_tokens=10):
    term_scores = tfidf_matrix[doc_index].toarray().flatten()
    important_term_indices = term_scores >= threshold
    
    # Filter terms based on threshold and preserve original order
    important_terms = [terms[i] for i in range(len(terms)) if important_term_indices[i]]
    
    # Retrieve original order from 'Notes Page Content2'
    original_text = df_for_Summary.at[doc_index, 'Azma_bart_tfidf'].split()
    important_terms_in_order = [term for term in original_text if term in important_terms]

    # Get the first 'top_tokens' tokens
    first_10_tokens = important_terms_in_order[:top_tokens]
    
    return ' '.join(first_10_tokens )

# Apply the function to each document in the DataFrame
df_for_Summary['Azma_bart_tfidf']= [
    get_important_terms(i, tfidf_matrix, terms, tfidf_threshold)
    for i in range(tfidf_matrix.shape[0])
]

df_for_Summary.head()

In [10]:
df_for_Summary.drop('Unnamed: 0',axis=1,inplace=True)
df_for_Summary.to_excel("Summarised_output_for_model.xlsx")

In [23]:
SSIC_ref_df = SSIC_ref_df.drop_duplicates(subset=['SSIC 2020'], keep='first')
SSIC_ref_df

,SSIC 2020,SSIC 2020 Title,Detailed Definitions,Cross References,Examples of Activities Classified Under this Code,"Section, 2 digit code",Division,Group,Class,Section,Section Title,Division Title,Group Title,Class Title
0,1111,Growing of leafy and fruit vegetables,This sub-class includes the cultivation of lea...,"•growing of mushrooms, see 01112\r\n•growing o...",NaN,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
1,1112,Growing of mushrooms,This sub-class includes the cultivation of mus...,•growing of leafy and fruit vegetables (non-hy...,NaN,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
2,1113,Growing of root crops,This sub-class includes the cultivation of roo...,•growing of leafy and fruit vegetables (non-hy...,NaN,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
3,1119,Growing of food crops (non-hydroponics) n.e.c.,This sub-class includes the propagation of foo...,•growing of leafy and fruit vegetables (non-hy...,•cereals growing (non-hydroponics)\r\n•rice gr...,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
4,1120,Growing of leafy and fruit vegetables (hydropo...,This sub-class includes the cultivation of lea...,•growing of leafy and fruit vegetables (non-hy...,NaN,1,1,11,112,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Hydroponics)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,97001,Activities of households as employers of domes...,This Sub-class includes activities of househol...,•activities of households as employers of othe...,NaN,97,97,970,9700,T,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,Activities of Households as Employers of Domes...
1019,97002,Activities of households as employers of other...,This Sub-class includes activities of househol...,•activities of households as employers of dome...,NaN,97,97,970,9700,T,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,Activities of Households as Employers of Domes...
1020,99010,Foreign embassies and trade representative off...,This Sub-class includes activities of: \r\n•di...,NaN,NaN,99,99,990,9901,U,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,Foreign Embassies and Trade Representative Off...
1021,99020,Foreign armed forces,This Sub-class includes activities of foreign ...,NaN,NaN,99,99,990,9902,U,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,Foreign Armed Forces


In [7]:
df_for_Summary = pd.read_excel("D:\ssicsync\LLM_Test\Summarised_output_for_model.xlsx", dtype = str)

Testing on prompts

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
list_model_name = ['llama3.1']

# Define the function to extract and summarize the business activity
def summarize_business_activity(content, model_name):
    response = ollama.chat(model=model_name, messages=[
        {
            'role': 'user',
            'content': f'''
            {content}

            
            extract the principal activity under "NOTES TO THE FINANCIAL STATEMENTS" sections in keywords only
            ''',
        },

    #extract 
    ])  # Specify the device
    response_v = response['message']['content']  # Get the response content
    return response_v

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# List of PDF files to process
pdf_directory = "D:/ssicsync/input_rawPDFReports/"
#pdf_files = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

#Test on certain files only
specific_pdf_files = [
    "ACCRELIST LTD. 2023 (201413466E).pdf",
    "CREATIVE TECHNOLOGY LTD. 2023 (198303359D).pdf",
    "NANOFILM TECHNOLOGIES INTERNATIONAL LIMITED 2022 (199902564C).pdf",
    "ABR HOLDINGS LIMITED 2022 (197803023H).pdf",
    "UNITED OVERSEAS INSURANCE LIMITED 2022 (197100152R).pdf"
]
pdf_files = [os.path.join(pdf_directory, f) for f in specific_pdf_files]

# DataFrame to store results
df_results = pd.DataFrame(columns=['PDF File', 'Extracted Text'] + [f'llm_output_{model}' for model in list_model_name])

#
# Process each PDF file
for pdf_file in pdf_files:
    extracted_text = extract_text_from_pdf(pdf_file)
    result_row = {'PDF File': pdf_file, 'Extracted Text': extracted_text}
    
    for model in list_model_name:
        column_name = 'llm_output_' + model
        result_row[column_name] = summarize_business_activity(extracted_text, model)
    
    df_results = df_results.append(result_row, ignore_index=True)

pd.set_option('display.max_colwidth', None)
# Save the results to an Excel file if needed
# df_results.to_excel('summarized_output.xlsx', index=False)
df_results['PDF Name'] = df_results['PDF File'].apply(lambda x: os.path.basename(x))
df_results.loc[:,['PDF Name', 'llm_output_llama3.1']].head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\4137931732.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(result_row, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\4137931732.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(result_row, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\4137931732.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(result_row, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\4137931732.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_res

,PDF Name,llm_output_llama3.1
0,ACCRELIST LTD. 2023 (201413466E).pdf,"I don't see any section labeled ""NOTES TO THE FINANCIAL STATEMENTS"". However, I can try to extract the principle activity from other relevant sections. \n\nBased on the provided text, the company's name is ACCRELIST HOLDINGS LIMITED and the following keywords are related to their activities:\n\n* ""Technology""\n* ""Software""\n* ""Digital Media""\n\nThese keywords might be related to the company's principal activities, but without more context or specific information from a ""NOTES TO THE FINANCIAL STATEMENTS"" section, it is difficult to confirm."
1,CREATIVE TECHNOLOGY LTD. 2023 (198303359D).pdf,"Based on the provided information, I was unable to find a section titled ""NOTES TO THE FINANCIAL STATEMENTS"". However, I can help you extract the principle activity from other available information.\n\nFrom the ""CORPORATE INFORMATION"" section:\n\n* Company Name: CREATIVE TECHNOLOGY LTD. AND ITS SUBSIDIARIES\n* Tagline: creative.com\n\nAssuming this is related to the company's principle activity:\n\n**Digital Technology / E-commerce**\n\nPlease note that I had to make an educated guess based on the provided information, and it may not be entirely accurate. If you have any additional context or details, please feel free to provide them!"
2,NANOFILM TECHNOLOGIES INTERNATIONAL LIMITED 2022 (199902564C).pdf,"Here is the extracted information:\n\n**""NOTES TO THE FINANCIAL STATEMENTS""**\n\n* Operating segment: **Manufacturing of coatings and surface technologies**\n* Principal activities:\n\t+ Development, manufacture and sale of various nanocoating products \n\t+ Supply of technical services to customers in connection with the use of its coating products"
3,ABR HOLDINGS LIMITED 2022 (197803023H).pdf,"Here is the extracted principle activity:\n\n""Property investment and property development, and provision of related services"""
4,UNITED OVERSEAS INSURANCE LIMITED 2022 (197100152R).pdf,Financial Services \nInsurance


In [30]:
list_model_name = ['llama3.1']

# Define the function to extract and summarize the business activity
def summarize_business_activity(content, model_name):
    response = ollama.chat(model=model_name, messages=[
        {
            'role': 'user',
            'content': f'''
            {content}

            
            extract the principal activity under "NOTES TO THE FINANCIAL STATEMENTS" sections in keywords only
            ''',
        },

    #extract 
    ])  # Specify the device
    response_v = response['message']['content']  # Get the response content
    return response_v

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# List of PDF files to process
pdf_directory = "D:/ssicsync/input_rawPDFReports/"
pdf_files = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# DataFrame to store results
df_results = pd.DataFrame(columns=['PDF File', 'Extracted Text'] + [f'llm_output_{model}' for model in list_model_name])

#
# Process each PDF file
for pdf_file in pdf_files:
    extracted_text = extract_text_from_pdf(pdf_file)
    result_row = {'PDF File': pdf_file, 'Extracted Text': extracted_text}
    
    for model in list_model_name:
        column_name = 'llm_output_' + model
        result_row[column_name] = summarize_business_activity(extracted_text, model)
    
    df_results = df_results.append(result_row, ignore_index=True)

pd.set_option('display.max_colwidth', None)
# Save the results to an Excel file if needed
# df_results.to_excel('summarized_output.xlsx', index=False)
df_results['PDF Name'] = df_results['PDF File'].apply(lambda x: os.path.basename(x))
df_results.loc[:,['PDF Name', 'llm_output_llama3.1']].head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\799553894.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(result_row, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\799553894.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(result_row, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\799553894.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(result_row, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8900\799553894.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results

,PDF Name,llm_output_llama3.1
0,ABR HOLDINGS LIMITED 2022 (197803023H).pdf,Investment Holding Company
1,ABUNDANCE INTERNATIONAL LIMITED 2022 (197501572K).pdf,Investment Holding Company
2,ABUNDANTE LIMITED 2023 (197902587H).pdf,Here are the principal activities extracted from the text:\n\n* Property investment and development\n* Hotel operation\n* Management of hotels and other properties\n* Trading and investment activities\n* Financing and investment activities
3,ACCRELIST LTD. 2023 (201413466E).pdf,Here are the extracted principal activities in keywords only:\n\n* Provision of information and consulting services
4,ACESIAN PARTNERS LIMITED 2022 (199505699D).pdf,"No principal activity mentioned in the provided text. \n\nHowever, I can try to help you with other information extraction tasks if needed!"


In [31]:
merged_df = pd.merge(df_for_Summary, df_results[['PDF Name', 'llm_output_llama3.1']], left_on='PDF Name', right_on='PDF Name', how='left')
#merged_df.to_excel("D:\ssicsync\LLM_Test\Summarised_output_for_model_v3.xlsx", index=False)

In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
import re

# Download stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

# Preprocess text function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    words_to_remove = ['principal', 'activity', 'activities', 'investment', 'holding', 'holdings', 'singapore', 'exchange', 'securities', 'trading', 'limited', 'company','extracted','keywords']  # List of additional words to remove
    # Remove additional words
    text = ' '.join(word for word in text.split() if word not in words_to_remove)
    return text

# Apply preprocessing to the 'llm_output_llama3.1' column
merged_df['llm_output_llama3.1_tfidf'] = merged_df['llm_output_llama3.1'].apply(preprocess_text)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed text
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['llm_output_llama3.1_tfidf'])

# Get feature names (terms)
terms = tfidf_vectorizer.get_feature_names_out()

# Set a TF-IDF threshold
tfidf_threshold = 0.1

# Function to get important terms from a document
def get_important_terms(doc_index, tfidf_matrix, terms, threshold, top_tokens=10):
    term_scores = tfidf_matrix[doc_index].toarray().flatten()
    important_term_indices = term_scores >= threshold
    
    # Filter terms based on threshold and preserve original order
    important_terms = [terms[i] for i in range(len(terms)) if important_term_indices[i]]
    
    # Retrieve original order from 'llm_output_llama3.1_tfidf'
    original_text = merged_df.at[doc_index, 'llm_output_llama3.1_tfidf'].split()
    important_terms_in_order = [term for term in original_text if term in important_terms]

    # Get the first 'top_tokens' tokens
    first_10_tokens = important_terms_in_order[:top_tokens]
    
    return ' '.join(first_10_tokens)

# Apply the function to each document in the DataFrame
merged_df['llm_output_llama3.1_tfidf'] = [
    get_important_terms(i, tfidf_matrix, terms, tfidf_threshold)
    for i in range(tfidf_matrix.shape[0])
]

# Check for empty TF-IDF outputs and replace with original content if empty
merged_df['llm_output_llama3.1_tfidf'] = merged_df.apply(
    lambda row: row['llm_output_llama3.1'] if not row['llm_output_llama3.1_tfidf'] else row['llm_output_llama3.1_tfidf'],
    axis=1
)

# Display the modified DataFrame
merged_df.loc[:,['llm_output_llama3.1_tfidf', 'llm_output_llama3.1']]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,llm_output_llama3.1_tfidf,llm_output_llama3.1
0,Investment Holding Company,Investment Holding Company
1,Investment Holding Company,Investment Holding Company
2,text property development hotel operation management hotels properties financing,Here are the principal activities extracted from the text:\n\n* Property investment and development\n* Hotel operation\n* Management of hotels and other properties\n* Trading and investment activities\n* Financing and investment activities
3,provision information consulting services,Here are the extracted principal activities in keywords only:\n\n* Provision of information and consulting services
4,mentioned provided text however try help information extraction tasks needed,"No principal activity mentioned in the provided text. \n\nHowever, I can try to help you with other information extraction tasks if needed!"
...,...,...
82,provided text telecommunications services equipment rental provision communication network infrastructure,Here are the principal activities extracted from the provided text:\n\n* Telecommunications services and equipment rental\n* Provision of communication network infrastructure
83,manufacturing electronic components,Manufacturing & Trading of Electronic Components
84,insurance providing general insurance reinsurance business general insurance business life,Insurance services \n\nproviding general insurance \n\nreinsurance business \n\ngeneral insurance business \n\nlife reinsurance business \n\ninvestment activities
85,notes financial statements section provision services goods manufacturing precision components,"Here are the principal activities extracted from the ""NOTES TO THE FINANCIAL STATEMENTS"" section in keywords only:\n\n* Provision of services\n* Trading in goods\n* Manufacturing and trading of precision components"


In [36]:
merged_df.to_excel("D:\ssicsync\LLM_Test\Summarised_output_for_model_v3.xlsx", index=False)

KeyBert
https://github.com/MaartenGr/KeyBERT

Embedding documents; 
Creating candidate keywords; 
Calculating best keywords through either MMR, Max Sum Similarity, or Cosine Similarity

In [77]:
from keybert import KeyBERT

# description = """principal activities of the Company are the prefabrication of steel reinforcement for use in concrete, 
# trading of steel reinforcing bars, and manufacturing and sale of wire mesh fences. 
# The principal activities of the subsidiaries are disclosed in Note 13 to the financial statements. 
# """
description = """principal activities of the Company are those of print and paper management related activities and investment holding. 
The principal activities of the subsidiaries are those of trading of commodity chemical products, 
provision of water treatment solutions using microbial and/or chemicals in the People s Republic of China ( PRC ), 
print and paper management related activities and in investment and trading of securities.
"""

#keyphrase_ngram_range ---->  depending on the number of words in the resulting keyphrases
keywords = KeyBERT().extract_keywords(description,keyphrase_ngram_range=(2, 2),top_n=10)
keywords

[('principal activities', 0.6735),
 ('activities subsidiaries', 0.5877),
 ('activities company', 0.4743),
 ('subsidiaries trading', 0.4476),
 ('activities investment', 0.409),
 ('holding principal', 0.3914),
 ('chemical products', 0.3815),
 ('company print', 0.3769),
 ('paper management', 0.3719),
 ('related activities', 0.3711)]

In [ ]:
# apply keybert on ref table, change the range accordinly
SSIC_ref_df['Keywords'] = SSIC_ref_df['Detailed Definitions'].apply(
    lambda x: kw_model.extract_keywords(x, keyphrase_ngram_range=(2, 2))
)

SSIC_ref_df['Keywords']
#return: keywords: The top n keywords for a document with their respective distancesto the input document.
SSIC_ref_df.to_excel("SSIC Fact Ref_with keywords.xlsx")

In [63]:
#compare keywords got from description
for _, row in SSIC_ref_df.iterrows():
    fact_keywords = [kw[0] for kw in row['Keywords']]
    new_submission_keywords = [kw[0] for kw in keywords]
    match = set(fact_keywords).intersection(set(new_submission_keywords))
    
    if match:
        print(f"Matching SSIC: {row['SSIC 2020']}, Keywords: {match}")

Embedding comparison

In [67]:
from sentence_transformers import SentenceTransformer, util

#pretained minil,
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert the summarized descriptions and SSIC definitions to embeddings
test_linkedin=df_linkedIn.head(10)
test_ssicref=SSIC_ref_df.head(10)
test_linkedin['Embeddings'] = (test_linkedin.head(10))['Summarized_Description'].apply(lambda x: model.encode(x))
test_ssicref['Embeddings'] = test_ssicref['Detailed Definitions'].apply(lambda x: model.encode(x))


def find_best_ssic_match(company_embedding, fact_table):
    best_match = None
    best_similarity = -1
    best_definition = None

    for _, row in fact_table.iterrows():
        similarity = util.pytorch_cos_sim(company_embedding, row['Embeddings'])[0][0].item()
        
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = row['SSIC 2020']
            best_definition = row['Detailed Definitions']
    
    return best_match, best_similarity, best_definition

# Find the best matching SSIC code and its detailed definition for each company
matches = test_linkedin['Embeddings'].apply(lambda x: find_best_ssic_match(x, test_ssicref))

# Extract the best match SSIC and detailed definition
test_linkedin['Best_Match_SSIC'] = matches.apply(lambda x: x[0])
test_linkedin['Best_Match_Definition'] = matches.apply(lambda x: x[2])

test_linkedin

C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3277016640.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_linkedin['Embeddings'] = (test_linkedin.head(10))['Summarized_Description'].apply(lambda x: model.encode(x))
C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3277016640.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ssicref['Embeddings'] = test_ssicref['Detailed Definitions'].apply(lambda x: model.encode(x))
C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3277016640.py:32: Setting

,PDF Name,Page Number,Notes Page Content,Summarized_Description,Embeddings,Best_Match_SSIC,Best_Match_Definition
0,ABR HOLDINGS LIMITED 2022.pdf,74,principal activities of the Company are the ma...,The principal activities of the Company are th...,"[-0.020132285, -0.10326839, -0.01515015, 0.012...",1120,This sub-class includes the cultivation of lea...
1,ABUNDANCE INTERNATIONAL LIMITED 2022.pdf,56,principal activities of the Company are those ...,The principal activities of the Company are th...,"[-0.033344787, -0.049288258, 0.022993648, -0.0...",1190,This Sub-class includes the growing of other c...
2,ABUNDANTE LIMITED 2023.pdf,48,principal activities of the Company are those ...,The Company's ultimate controlling party is Le...,"[-0.07526265, -0.040298507, 0.029111238, 0.032...",1111,This sub-class includes the cultivation of lea...
3,ACCRELIST LTD. 2023.pdf,55,principal activities of the Company is investm...,The principal activities of the Company are in...,"[-0.0043711183, -0.050210413, -0.010950963, -0...",1190,This Sub-class includes the growing of other c...
4,ACESIAN PARTNERS LIMITED 2022.pdf,54,principal activities of the Group consist of d...,The principal activities of the Group consist ...,"[-0.0027161513, 0.0018759626, -0.027635382, -0...",1190,This Sub-class includes the growing of other c...
5,ACMA LTD. 2022.pdf,48,principal activity of the Company is that of i...,The principal activity of Acma Ltd. is that of...,"[-0.046834115, 0.02470865, -0.029877186, -0.02...",1190,This Sub-class includes the growing of other c...
6,ADVANCED SYSTEMS AUTOMATION LIMITED 2022.pdf,52,principal activity of the Company is investmen...,The principal activity of the Company is inves...,"[-0.07500034, -0.021864938, -0.04694008, 0.070...",1190,This Sub-class includes the growing of other c...
7,ALLIANCE HEALTHCARE GROUP LIMITED 2023.pdf,82,principal activities of the company is that of...,The principal activities of the company are in...,"[-0.06722844, -0.008623031, -0.03112161, -0.01...",1190,This Sub-class includes the growing of other c...
8,ANNAIK LIMITED 2022.pdf,71,principal activity of the Company is that of i...,The principal activity of the Company is that ...,"[-0.029021166, -0.020046683, 0.020063434, -0.0...",1190,This Sub-class includes the growing of other c...
9,AP OIL INTERNATIONAL LIMITED 2022.pdf,61,The The Company is an investment holding comp...,The Company is an investment holding company. ...,"[-0.05421597, -0.0050037336, 0.006176454, 0.00...",1130,This Sub-class includes growing of fruits such...


In [74]:
test_row_index = 1  
test_description = test_linkedin.iloc[test_row_index]['Notes Page Content']
test_summary = summarizer(test_description, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
print("Summary:", test_summary)

Your max_length is set to 150, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


Summary: The principal activities of the Company are those of print and paper management related activities and investment holding. The subsidiaries include trading of commodity chemical products, provision of water treatment solutions using microbial and/or chemicals in the People's Republic of China and in investment and trading of securities.


In [75]:
test_embedding = model.encode(test_summary)
SSIC_ref_df['Embeddings'] = SSIC_ref_df['Detailed Definitions'].apply(lambda x: model.encode(x))

def find_top_ssic_matches(test_embedding, fact_table_embeddings):
    similarities = []

    for _, row in fact_table_embeddings.iterrows():
        similarity = util.pytorch_cos_sim(test_embedding, row['Embeddings'])[0][0].item()
        similarities.append((row['SSIC 2020'], similarity, row['Detailed Definitions']))

    topN = sorted(similarities, key=lambda x: x[1], reverse=True)[:3] #n=3
    return topN

top_matches = find_top_ssic_matches(test_embedding, SSIC_ref_df)

for match in top_matches:
    print(f"SSIC Code: {match[0]}, Similarity: {match[1]}, Definition: {match[2]}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3155042889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SSIC_ref_df['Embeddings'] = SSIC_ref_df['Detailed Definitions'].apply(lambda x: model.encode(x))


SSIC Code: 94110, Similarity: 0.4943753480911255, Definition: This Sub-class includes:
•activities of organisations whose members’ interests centre on the development and prosperity of enterprises in a particular line of business or trade, including farming, or on the economic growth and climate of a particular geographical area or political sub-division without regard for the line of business
•activities of federations of such associations
•activities of chambers of commerce, guilds and similar organisations
•dissemination of information, representation before government agencies, public relations and labour negotiations of business and employer organisations
SSIC Code: 64201, Similarity: 0.49186697602272034, Definition: This Sub-class includes the activities of bank/financial holding companies, i.e. units that hold the assets (owning controlling-levels of equity) of a group of subsidiary corporations and whose principal activity is owning the group. The holding companies in this Sub-

SSIC Prediction


In [ ]:
import pandas as pd
df_summarised=pd.read_excel("Summarised_output_new.xlsx")
df_summarised.drop('Unnamed: 0',axis=1,inplace=True)
df_summarised

In [ ]:
# Remove duplicates based on the 'SSIC 2020' column, keeping the first occurrence
SSIC_ref_df = SSIC_ref_df.drop_duplicates(subset=['SSIC 2020'], keep='first')

In [ ]:
#get embeddings for SSIC ref table
def get_embeddings(text,tokenizer,model):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

SSIC_ref_df['Embeddings'] = SSIC_ref_df['Detailed Definitions'].apply( lambda x: get_embeddings(x,tokenizer,model).numpy())

df_for_Summary['original_embedding']=df_for_Summary['Notes Page Content'].apply(lambda x: get_embeddings(x,tokenizer,model).numpy())

In [13]:
#predict SSIC
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader
from transformers import pipeline
import tensorflow as tf
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=87) # num_labels=len(df_prep)

# Prediction pipeline
predictor = pipeline('text-classification', model=model, tokenizer=tokenizer,framework='pt', device=0 if torch.cuda.is_available() else -1)

# Example company description
company_description = "The consolidated financial statements relate to the Company and its subsidiaries. The Group acquired 100% of the ordinary shares Fu Yu Supply Chain Solutions Pte. Ltd. in prior year. FYSCS is engaged in the business in providing supply chain management services.."
predictions = predictor(company_description)
predicted_ssic_code = predictions[0]['label']

print(f"Predicted SSIC Code: {predicted_ssic_code}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted SSIC Code: LABEL_62


In [15]:
lvl_train = 'SSIC 2020'
lvl_train_title = lvl_train + " Title"
###############################################################################################################################################

df_prep = SSIC_ref_df[[lvl_train, lvl_train_title, 'Detailed Definitions']]
df_prep['encoded_cat'] = df_prep[lvl_train].astype('category').cat.codes
# df_prep['encoded_cat'] = df_prep[lvl_train] + ' ' + df_prep[lvl_train_title]
# df_prep['encoded_cat'] = df_prep['encoded_cat'].astype('category')

data_texts = df_prep['Detailed Definitions'].to_list() # Features (not tokenized yet)
data_labels = df_prep['encoded_cat'].to_list() # Labels

df_prep = df_prep[[lvl_train, 'encoded_cat']].drop_duplicates()
df_prep

C:\Users\Admin\AppData\Local\Temp\ipykernel_23540\1649197315.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep['encoded_cat'] = df_prep[lvl_train].astype('category').cat.codes


,SSIC 2020,encoded_cat
0,1111,0
1,1112,1
2,1113,2
3,1119,3
4,1120,4
...,...,...
1018,97001,1018
1019,97002,1019
1020,99010,1020
1021,99020,1021


In [18]:
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch

# Assuming data_texts and data_labels are defined
train_texts = data_texts
train_labels = data_labels
val_texts = data_texts
val_labels = data_labels

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0, shuffle=True)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1023)
model.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, eps=1e-08)

for epoch in range(3):  # Assuming 3 epochs
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 